RCA uses a network output to train a classifier which can be applied to other labelled data. This notebook attempts to use random forest segmentation to achieve this.
THIS STUFF WILL ONLY WORK ON IMAGES, NOT VOLUMES.

In [ ]:
from tensorflow.keras.models import model_from_json,clone_model

from mask_utils import iou,dsc,mean_contour_distance,symmetric_hausdorff_distance,show_image_with_masks

from network_utils import augmentImageSequence, gpu_memory_limit

import numpy as np

import os

from sklearn.model_selection import train_test_split

import copy

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier

from scipy import signal

from scipy.stats import pearsonr

In [ ]:
gpu_memory_limit(8000)

Load data and model, and get them sorted in the same way as in the notebooks used to train models (i.e. same train/test split, random seed etc)

In [ ]:
DataDir = './data/pericardial/wsx_round2/'

#load data - these files created by extract_dcm_for_wsx.ipynb
X = np.load(os.path.join(DataDir,'X.npy'))
Y = np.load(os.path.join(DataDir,'Y.npy')).astype('float')
pxArea = np.load(os.path.join(DataDir,'pxSize.npy'))
pxSpacing = np.sqrt(pxArea)

#ensure the shape is correct arrays saved were rank 3, so this changes to rank 4 (last dimension represents channels)
X = X.reshape([*X.shape,1])
Y = Y.reshape([*Y.shape,1])

#do train/test split!
X_train, X_test, Y_train, Y_test,pxArea_train,pxArea_test,pxSpacing_train,pxSpacing_test = train_test_split(X, Y, pxArea,pxSpacing, test_size=0.2,random_state=101)

# X = X[:200,:,:,:]
# Y = Y[:200,:,:,:]

#
# M = X.shape[0]
# MTest = X_test.shape[0]

Now, need to load a model which can be used for the RCA

In [ ]:
#pick a model, just need one to play with.
modelBaseName = './data/models/mrunet_2020-04-07_09:59' #THIS MODEL IS NOT THE BEST ONE BUT HAS BEEN SELECTED TO GIVE A WIDE SPREAD IN IOU ON TRAIN AND TEST SETS

#load the model archistecture
with open( modelBaseName + '.json', 'r') as json_file:
    model = model_from_json( json_file.read() )
    
#get the weights
model.load_weights(modelBaseName + '.h5')    


#make predictions on the test set:
Pred_test = model.predict(X_test) > 0.5

In [ ]:
def cuboid_mean(image,cuboidRadii):

    '''Takes the mean of 
    image is an image. 
    cuboidRadii is a tuple of ints, representing the radii of 
    actually the rectangular mean, as this will only work on images (not volumes or hypervolumes).'''
    
    #pad the image with edge pixels so that edges don't tend to 0 after convolution
    pads = [(d,d) for d in cuboidRadii] # convert radii to symmetric tuples for each dimension
    image = np.pad(image,pad_width=(pads),mode='edge')
    
    dim = [1+2*d for d in cuboidRadii] #only odd edge lengths allowed, as centers must be unambiguous
    
    #create block for convolution
    block = np.ones(dim,dtype='float')
    
    #do the convolution
    cuboidMean = signal.convolve(image,block,method='auto',mode='valid') #valid convolution will reduce size of array back to original dims.
    
    return cuboidMean
    
def cuboid_offset_mean_difference(image,cuboidRadii,offsets):
    
    '''nonlocal feature generation - takes the cuboid mean (actually rectangular mean) and subtracts it from each pixel, but offset by some dimensions specified by offset (which should contain x and y)'''
    
    #pad the edges using the offsets - which must be on the correct side i.e. before if negative, after if positive. This bit will work for higher-dimensions!
    pads = []

    for offset in offsets:
        if offset < 0:
            pads.append((abs(offset),0))
        else:
            pads.append((0,offset))
    
    image_padded = np.pad(image,pad_width=pads,mode='edge')
    
    #calculate the cuboid mean
    cuboidMean = cuboid_mean(image_padded, cuboidRadii)
        
    #keep the bit corresponding to the original shape, allows elementwise subtractions
    indices = []
    for dim,offset in zip(cuboidMean.shape,offsets):
        if offset < 0:
            indices.append(slice(-dim,offset))
        else:
            indices.append(slice(offset,dim))
            
    cuboidMean = cuboidMean[tuple(indices)]
    
    cuboidMeanDifference = image - cuboidMean
    
    return cuboidMeanDifference

def image2features(image):
    
    '''take an image (of arbitrary dimension) and convert it to an array of (npixels,nfeatures)'''
    
    npx = np.product(image.shape[:-1])
    
    nfeatures = image.shape[-1]
    
    features = image.reshape(npx,nfeatures)
    
    return features

def feature_engineer_function(nFeatures=500,maxRadius=4,maxOffset=8,random_seed = None,prior = None):
    
    '''this function RETURNS A FUNCTION which can be used to process images into (npixels,nfeatures)
    default arguments: Zikic et al allow a max radius of 5mm (approx 3px in my dataset), and a max offset of 15mm (approx 8px in my dataset).
    Valindria et al use 10000 features, but their problem is 3D. 10000^(2/3) = 464, so perhaps a default of 500 is appropriate for a 2D problem?????????
    also allows the addition of precomputed priors
    '''
    
    
    #calculate the maximum number of unique parameter sets that can be generated with the maximum parameters specified.
    maxParameterSets = maxRadius**2 * (maxOffset*2 - 1)**2
    
    assert maxParameterSets > nFeatures,'you are trying to generate more features than are mathematically possible (max is ' + str(maxParameterSets) + ')'
    
    #set random state so that deterministic behaviour can be guaranteed if necessary
    if random_seed is not None:
        np.random.seed(random_seed)
    
    #generate a big set of parameters that can be fed into 
    radii = np.random.randint(low=0,high=maxRadius,size=(nFeatures,2))
    offsets = np.random.randint(low=-maxRadius+1,high=maxRadius,size=(nFeatures,2))

    def feature_Function(image):
        
        #only need to use the cuboid_offset_mean_difference as cuboid_mean is a special case of this (with offsets = [0,0])    

#         [image.squeeze()]
        feat = np.stack([cuboid_offset_mean_difference(image.squeeze(),radius,offset) for radius,offset in zip(radii,offsets)],axis=-1)

        if prior is not None:
            feat = np.concatenate([feat,prior],axis=-1)
        
        feat = image2features(feat)
        
        return feat
    
    return feature_Function

In [ ]:
class preprocess_and_predict_model_wrapper():
    
    '''this class exists to allow all preprocessing of images to be incorporated into an object used for '''
    
    
    def __init__(self,model,preprocessing_function):
        
        self.model = model #this should be, for example, an sklearn model instance
        self.preprocessing_function = preprocessing_function #a function which preprocesses inputs for use in the model
        
    def predict(self,x):

        #preprocess the input (which is likely an image)
        xProcessed = self.preprocessing_function(x)

        #actually make the prediction
        y = self.model.predict(xProcessed)

        #convert the output back into the image shape, preserving channels if they exist
        y = y.reshape(*x.shape[:-1],-1)

        return y


def train_rca_model(x,y,featureOptions={},forestOptions={}):
    
    '''optional dictionaries for feature engineering and random forest'''
#     assert x.shape[0] == 1 and y.shape[0]==1, 'you can only do RCA on one image at a time! or alternatively the images are the wrong shape'
    
    #threshold mask so that it can be used as a target classifier
    y = y > 0.5
    
    #generate feature engineering function which can be reused
    feature_Function = feature_engineer_function(**featureOptions)
       
    #reshape multidimensional image into (npx,nfeatures) array, and add features
    x = feature_Function(x)

    y = image2features(y).flatten() #FIXME this only works for cases with two output classes.
    
    rf = RandomForestClassifier(**forestOptions) #pass options in
    
    rf.fit(x,y)
    
    #now, wrap the function for preprocessing images and the model into a single object with a predict() method:
    rcaModel = preprocess_and_predict_model_wrapper(model=rf,preprocessing_function=feature_Function)
    
    return rcaModel

In [ ]:
def evaluate_model(model,X_val,Y_val):
    
    '''this function takes a model (presumably retrained on a predicted mask in order to do RCA) and evaluates it on the set of masks which are known'''

    #FIXME THIS IS FUCKING LAZY AND WILL FALL OVER REALLY EASILY
    try:
        Y_pred = model.predict(X_val)
    except:
        Y_pred = [model.predict(X) for X in X_val]
    
    ious = np.array([iou(Y_pred[m],Y_val[m]) for m in range(len(Y_pred))])
    
    #FIXME add some more metrics fam (mcd,hd,dsc)
    
    return ious

In [ ]:
def rca_evaluate(x,pred,X_val,Y_val,featureOptions={},forestOptions={}):
    
    assert np.all(model.input_shape[1:] == x.shape[1:]),'image input shape and model input do not match - have you reshaped the image correctly?'
    assert x.shape[0] == 1, 'you can only do RCA on one image at a time! or you might have reshaped image wrongly....'
    
    #mean of the evaluation output across the subjects - corresponds to voxelwise probability of each class. FIXME BE A BIT FUCKING CLEVERERER
    prior = np.mean(Y_val,axis=0)
    
    featureOptions['prior'] = prior
    
    rcaModel = train_rca_model(x,pred,featureOptions,forestOptions)
    
    ious = evaluate_model(rcaModel,X_val,Y_val)
    
    return ious#,mcd FIXME!!!! this should ultimately return only a single number/collection of values

First, we should look at the iou spread over the whole thing... The IOUs of all of the data we plan to use, to evaluate the *original* model

In [ ]:
trueIOUs = evaluate_model(model,X_test,Y_test)

plt.hist(trueIOUs,bins = np.arange(0,1.05,0.05))

Additionally, there are considerations about which evaluation set should be used. It seems to me that thw whole point of this is to do with looking for similarities to the training set - so this should be used to get the predictions.

Now, lets just get a single datapoint to play with... 

In [ ]:
MTest =  X_test.shape[0]
M = X_train.shape[0]


#select an example image
np.random.seed(21)
egInd = np.random.randint(X_test.shape[0])

#get the IOU that we want to predict...
trueIOU = trueIOUs[egInd]

#get the actual image out and shaped correctly
egX = X_test[egInd,:,:].reshape(1,*model.input_shape[1:])

egPred = Pred_test[egInd,:,:].reshape(1,*model.input_shape[1:])

#select a subset of training set images for quicker evaluation of the RCA method
MVal = 50
sel4val = np.random.randint(0,M,MVal)

X_val = X_train[sel4val,:,:,:]
Y_val = Y_train[sel4val,:,:,:]


In [ ]:
rcaModel = train_rca_model(egX,egPred)

negs = 25

egs = np.random.choice(range(MVal), negs, replace=False)

ncols = 5
nrows = np.ceil(negs/ncols)

plt.figure(figsize = (5*ncols,5*nrows))

imShape = X.shape[1:-1]

#FIRST show the real segmentation i.e. MANUAL, MACHINE AND RCA 
plt.subplot(nrows,ncols,1)
    
manualMask,rcaMask = egY.reshape(imShape), rcaModel.predict(egX.reshape(imShape))

show_image_with_masks(image = egX.reshape(imShape),
                      masks = [manualMask,predY.reshape(imShape),rcaMask],
                      maskOptions = [{'linewidth':1,'color':'g'},{'linewidth':1.5,'color':'b'},{'linewidth':1,'color':'r'}]
                     )

plt.title('ORIGINAL IMAGE')

for i in range(1,negs):
    
    plt.subplot(nrows,ncols,i+1)
    
    manualMask,rcaMask = Y_val[egs[i]].reshape(imShape) > 0.5, rcaModel.predict(X_val[egs[i]].reshape(imShape))
    
    pxS = pxSpacing[egs[i]]

    
    show_image_with_masks(image = X_val[egs[i],:,:].reshape(imShape),
                          masks = [manualMask,rcaMask],
                          maskOptions = [{'linewidth':1,'color':'g'},{'linewidth':1,'color':'r'}]
                         )
    
    plt.title('iou = ' + f'{iou(manualMask,rcaMask):.03}' + '\n' + 
              'hd = ' + f'{symmetric_hausdorff_distance(manualMask,rcaMask,pxS):.03}' + '\n' +
              'mcd = ' + f'{mean_contour_distance(manualMask,rcaMask,pxS):.03}')


In [ ]:

predictedIOUs = rca_evaluate(egX,egPred,X_val,Y_val)

plt.figure(figsize = (10,5))


plt.subplot(1,2,1)
plt.hist(predictedIOUs,bins= np.arange(0,1.05,0.05),density=True,label = 'IOU of evaluation examples after retraining')
plt.plot([trueIOU,trueIOU],plt.ylim(),c='r',label = 'true IOU')
plt.xlabel('iou')
plt.ylabel('probability density')
plt.legend()


plt.subplot(1,2,2)
plt.scatter(evaluate_model(model,X_val,Y_val),predictedIOUs,label = 'evaluation set')
plt.xlim([0,1])
plt.ylim([0,1])

plt.plot([0,1],[trueIOU,trueIOU],c='r',label = 'true IOU')
plt.plot([0,1],[0,1],c='k',label = 'line of unity')
plt.xlabel('original IOU')
plt.ylabel('IOU after retraining')

plt.legend()


In [ ]:
#get the true IOU for all of the test set
trueIOUs = evaluate_model(model,X_test,Y_test)

predIOUs = np.zeros((MTest,MVal))

#loop over each test set example
for ind in range(MTest):

    egX = X_test[ind,:,:,:].reshape(1,208,208,1)
    
    predIOUs[ind,:] = rca_evaluate(egX,egPred,X_val,Y_val,featureOptions,forestOptions)
    

In [ ]:
plt.figure(figsize = (5,15))

y = np.max(predIOUs,axis=1)
plt.subplot(3,1,1)
plt.plot([0,1],[0,1],c='k')
plt.scatter(trueIOUs,y)
plt.title( 'r = ' + f'{pearsonr(trueIOUs,y)[0]:.02}') 
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('predicted IOU (max)')                      

y = np.median(predIOUs,axis=1)            
plt.subplot(3,1,2)¬
plt.plot([0,1],[0,1],c='k')
plt.scatter(trueIOUs,y)
plt.title('r = ' + f'{pearsonr(trueIOUs,y)[0]:.02}')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('predicted IOU (median)')                      
  
y = np.mean(predIOUs,axis=1)
plt.subplot(3,1,3)
plt.plot([0,1],[0,1],c='k')
plt.scatter(trueIOUs,y)
plt.title('r = ' + f'{pearsonr(trueIOUs,y)[0]:.02}')
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('predicted IOU (mean)')                      

            
plt.xlabel('true IOU')


So, that doesn't work as well as could be hoped. Lets do grid search over the whole thing? You never know....

In [ ]:
featureOptions = dict(nFeatures=500,
                      maxRadius=4,
                      maxOffset=8,
                      random_seed=None
                     )

In [ ]:
forestOptions = dict(n_estimators=50,
                     criterion='gini', #Zikic et al use entropy
                     max_depth=30,
                     class_weight='balanced',#balance class weights as this problem is imbalanced (as are most semantic segmentation tasks...)
                     n_jobs=None, #don't be a dick
#                      random_state=101, #determinism
                     min_samples_split = 10,
                     max_features = 'auto',
                     
                    )

In [ ]:
#list of all parameters and their values for a fuck-off grid search
nFeatures = [50,100,500,1000]
maxRadius = [5,10,15,20]
maxOffset = [5,10,15,20]

n_estimators = [30,50,100,200,500]
max_features = ['auto','sqrt']
bootstrap = [True,False]
criterion = ['gini','entropy']
min_samples_split = [3,10,30,100]
min_samples_leaf = [1,3,10,30,100]
max_depth = [10,30,100]

In [ ]:
def get_feature_dict(params):
    
    featureOptions = {'random_seed':304} #DETERMINISM
    
    featureOptions['nFeatures'],featureOptions['maxRadius'],featureOptions['maxOffset'] = params
    
    return featureOptions
    
featureParams = [p.flatten() for p in np.meshgrid(nFeatures,maxRadius,maxOffset)]

featureDicts = [get_feature_dict(p) for p in zip(*featureParams)]

print(len(featureDicts))

In [ ]:
def parallelize_feature_eng(image,paramDict):
    
#     paramDict['prior'] = np.mean(Y_val,axis=0)
    
    func = feature_engineer_function(**paramDict)
    
    return func(image)

In [ ]:
X_test.shape

In [ ]:
#parallelise preprocessing of both test set and evaluation set

MTest = 50
X_test = X_test[:MTest]

from multiprocessing import Pool
import pickle

X_test_features = []
X_val_features = []

with Pool(processes=4) as p:
    
    for i,f in enumerate(featureDicts):
        
        feats =  p.starmap(parallelize_feature_eng,zip(X_test,[f]*MTest ))
        name = './Xfeat' + str(i) + '.pickle'
        X_test_features.append(name)
        pickle.dump(feats,open(name,'wb'))
        
        feats =  p.starmap(parallelize_feature_eng,zip(X_val,[f]*MVal )) 
        name = './Xval' + str(i) + '.pickle'
        X_val_features.append(name)
        pickle.dump(feats,open(name,'wb'))

In [ ]:
def get_forest_dict(params):
    
    forestOptions = {'class_weight':'balanced'}#balance class weights as this problem is imbalanced (as are most semantic segmentation tasks...)
    
    forestOptions['n_estimators'],forestOptions['max_features'],forestOptions['bootstrap'],forestOptions['criterion'],forestOptions['min_samples_split'],forestOptions['min_samples_leaf'],forestOptions['max_depth'] = params
    
    return forestOptions
    
#all parameter sets as dictionaries
forestParams = [p.flatten() for p in np.meshgrid(n_estimators,max_features,bootstrap,criterion,min_samples_split,min_samples_leaf,max_depth)]
forestParams = [get_forest_dict(d) for d in list(zip(*forestParams))]


In [ ]:
#get all combinations of feature/function indices and forest parameters
featureIndices = range(len(featureDicts))
allParams = list(zip(*[x.flatten() for x in np.meshgrid(featureIndices,forestParams)]))

In [ ]:
def get_r_from_params(params):
    
    #extract hyperParameters
    featureIndex,forestOptions = params

    #get the processed images using featureIndex
    ims_preprocessed = pickle.load(open(X_test_features[featureIndex],'rb')
    
    #now use featureIndex to get the preprocessed evaluation images
    X_val_preprocessed = pickle.load(open(X_val_features[featureIndex],'rb')
    
    #loop over the processed images
    predIOUs = []
    
    for m in range(MTest):
        
        imageFeatures = ims_preprocessed[m]
                
        #get the flattened predictions using featureIndex
        pred = Pred_test[m].flatten()

        #instantiate and train the RCA model using forestOptions
        rca_model = RandomForestClassifier(**forestOptions)

        
        rca_model.fit(imageFeatures,pred)
        #how well does the model do during fitting?
#         print(iou(rca_model.predict(imageFeatures),pred))
        
        
        #calculate iou between RCA predictions and evaluation masks, and take the max, adding to list
        predIOUs.append( max([iou(rca_model.predict(x)>0.5,y.flatten()>0.5) for x,y in zip(X_val_preprocessed,Y_val)]) )        

    #get r between the max ious and the true ious
    r = pearsonr(predIOUs,trueIOUs[:len(predIOUs)])
    
    return r

In [ ]:
with Pool(processes=4) as p:
    correlations = p.map(get_r_from_params,allParams)
    pickle.dump(correlations,open('./correlations.pickle','rb'))
    
    r,p = map(np.array,zip(correlations))    

In [ ]:
plt.hist(r)